In [2]:
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
import warnings

In [ ]:
def dataExtractor(name):
    file_path = 'FluidsInput/FluidInput_' + name +'.mat'
    matlab_data = scipy.io.loadmat(file_path)
    # Extract the 'connectivity' field from the 'Data' structured array
    data_structure = matlab_data['Data']
    connectivity_raw = data_structure['connectivity'][0, 0]  # Access the data (adjust indexing if needed)
    # Reshape or ensure it's a proper 2D array (if required)
    connectivity_data = connectivity_raw.squeeze()
    # Create a DataFrame from the connectivity data
    connectivity_df = pd.DataFrame(connectivity_data, columns=['Parent', 'Daughter1', 'Daughter2', 'Daughter3'])
    # Save the DataFrame to inspect it
    return connectivity_df

In [5]:
name = 'PVB046_arteries'
connectivity = dataExtractor(name)
print(connectivity)

     Parent  Daughter1  Daughter2  Daughter3
0         0         90         93          0
1         1          0          0          0
2         2          0          0          0
3         3          0          0          0
4         4          0          0          0
..      ...        ...        ...        ...
166     166        112        167          0
167     167         70        135          0
168     168          1         44          0
169     169         32         81         87
170     170        153        160          0

[171 rows x 4 columns]
